In [22]:
import getpass
import os
from dotenv import load_dotenv

os.environ["LANGCHAIN_TRACING_V2"] = "true"

load_dotenv()
if "OPENAI_API_KEY" not in os.environ:
    os.environ["OPENAI_API_KEY"] = getpass.getpass("Enter your OpenAI API key: ")
if "LANGCHAIN_API_KEY" not in os.environ:
    os.environ["LANGCHAIN_API_KEY"] = getpass.getpass("Enter your LangChain API key: ")

In [ ]:
from typing import TypedDict
from langchain.chat_models import init_chat_model
from langchain_core.messages import AnyMessage
from langchain_core.messages.utils import count_tokens_approximately
from langgraph.graph import StateGraph, START, MessagesState
from langgraph.checkpoint.memory import InMemorySaver
from langmem.short_term import SummarizationNode, RunningSummary

# Initialize model
model = init_chat_model("openai:gpt-4o")
summarization_model = model.bind(max_tokens=128)

# Define state
class State(MessagesState):
    context: dict[str, RunningSummary]

class LLMInputState(TypedDict):
    summarized_messages: list[AnyMessage]
    context: dict[str, RunningSummary]

# Summarization node with correct field binding
summarization_node = SummarizationNode(
    token_counter=count_tokens_approximately,
    model=summarization_model,
    max_tokens=1000,
    max_tokens_before_summary=1000,
    max_summary_tokens=512,
).with_config(field="context")

# LLM call node
def call_model(state: LLMInputState):  
    response = model.invoke(state["summarized_messages"])
    return {"messages": [response]}

# Build graph
checkpointer = InMemorySaver()
builder = StateGraph(State)
builder.add_node("summarize", summarization_node)
builder.add_node("call_model", call_model)
builder.add_edge(START, "summarize")
builder.add_edge("summarize", "call_model")
graph = builder.compile(checkpointer=checkpointer)

# Run the graph with a consistent thread_id
config = {"configurable": {"thread_id": "1"}}
graph.invoke({"messages": "hi, my name is bob"}, config)
graph.invoke({"messages": "write a short poem about cats"}, config)
graph.invoke({"messages": "now do the same but for dogs"}, config)
final_response = graph.invoke({"messages": "what's my name?"}, config)

# Print final LLM message
final_response["messages"][-1].pretty_print()

# Access and print summary
thread_id = config["configurable"]["thread_id"]
summary_obj = final_response["context"][thread_id]
print("\nSummary:", summary_obj.summary)


================================== Ai Message ==================================

Your name is Bob.
Full response keys: dict_keys(['messages', 'context', 'summarized_messages'])
Context content: {}


KeyError: '1'